In [1]:
import llm as L
L.TOKEN_COUNT_PATH = '/data/ai_club/team_3_2024-25/tokcounts2/'

In [ ]:
'''
Hei! (Hi!)
Say hi to me in Finnish.

-> Hei

---

Minä olen Rossi. (I am Rossi.)
Rossi is my name. What's yours?
Tell me in Finnish.

-> Minä olen {Name}

---

Terve {Name}! (Hello {Name})!
You can click words to see pronunciations and translations. Looking at translations lets me know that you should review a word more in the future
Try looking up a translation.

---



'''

first_n_words = [
    (['hei'], exp=None, msg='Say hi to me in Finnish', expect='hei'),
    (['mina', 'olen'], exp=None, msg='Introduce yourself to me in Finnish', expect='mina olen <NAME>'),
    (['suomalainen', 'suomi'], etc...),
    (['sina', 'olet'], exp='<explain conj of verbs><explain that you can drop pronouns>', msg='tell me im finnish', expect=['sina olet suomalainen', 'olet suomalainen']),
    ...
    (['oletko', 'olenko', 'onko'], exp='<explain yes-no qs>', etc....),
    ....
    (['kyllä', 'ei'], exp='<etc>', msg='<GENERATE a yes/new question from prior words>', expect='<yes or no, but extra points if the user says more>')
    ...
    (['koira'], None, show_img=True) # some could show img to have user guess.
    ...
    (['-ni', 'sinun'], etc...) # introduce inflections in the same way, the inflection is added to some bot-chosen word via "generate a sentence with the inflection -ni"
    ...
    # its not clear how the llm will be able to use increasingly more words w/o vocab restriction
    # ^ idea: get random subset of words small enough for llm to work with and make sentence!
    # fields here:
    # - list of words to introduct
    # exp - optoinal explainer
    # msg - message or message format to show user after showing new words
    # exp - expected user answer
    # show_img - show img accompanying word (if possible?)
]

In [ ]:
'''
Hei! Minä olen Rossi (I am Rossi). Which of the words I said means "Hi" in Finnish? -> Hei

Kyllä (yes)! Now introduce yourself to me. -> Minä olen {NAME}

Your name is {NAME}? Respond in Finnish. -> Kyllä

<explainer: practice every day for 5 or more minutes. Click words to see info and translation. Seeing translation causes srs. You'll look up often, but less in future.>


'''

In [98]:
'''
Level 0 - hardcoded, only introduces a few words, explains, only a few messages
Level 1 - vocab restricted, 2 modes:
    1) SRS review mode (conversation is determined by srs words), and when that's done for the day ...
    2) introduce one word at a time from hardcoded list, some words include "explainers". User has to use the word themself to move on to next.
Level 2+ - vocab restricted (?) SRS review mode, and free conversation mode.

lvl 2 word introduction:
- introduce one word at a time (maybe exceptions, some have additional explanation, accompany image for some words).
- New words are introduced in a sentence using known words. 
- User must use the word for themself before seeing next

explainer examples -
    (olen known, introducing sinä olet )"olen vs olet vs on verb conj"
    (koira known, introducing posessive form in general using koira) "koira koirani possessive"
    (sinä and minä known) "sinä vs sinulla, minä vs minulla, noun case and its usage for possesive"
    "ystävälläsi onko koira -- combining inflections"

^ in some the user guesses answer, in others a translation is given
^ OR: user always guesses. Some are from prior knowledge, but others provide a picture (like koira)
'''

In [ ]:
'''
Level 0 needs to bring things up to simple question asking
'''

In [2]:
first_n_words = [
    ['hei', 'minä', 'olen']
]

In [191]:
# say you have these from level 0 already

allowed_words = [
    'hei',
    'minä',
    'sinä',
    'tämä'
    'mitä',
    'kuka',
    'kyllä',
    'olen',
    'olenko',
    'olet',
    'oletko',
    'on',
    'suomi',
    'koira',
    'mukava',

    'rossi'
]

In [212]:
l = L.LLM(f'You are a Finnish language teacher named Rossi.\n\nIMPORTANT: Your responses must only use words in this allowed vocab: {allowed_words} and any emoji/punctuation.')

def chat(msg):
    out = l(msg)
    for t in out:
        print(f'{t}', end='')

In [213]:
chat('Terve :) Keep this interactive by asking ME questions, but only using allowed vocab.')

Hei! 😊 Kuka sinä olet?

In [219]:
chat('kylla')

😊 Oletko sinä tämä?

In [148]:
# the "continue the sentence with one of these words" approach

l = L.LLM('You are a Finnish language teacher named Rossi.')
# l(f'Start a Finnish sentence with one of the words in this allowed vocab: {allowed_words}.', response_format=['word'])

l(f'Vocab: {allowed_words} and any punctuation. Start a sentence with one of the words in the vocab.')
# l(f'Vocab: {allowed_words} and any punctuation. Continue this sentence with one of the words in the vocab:\n\nHei', response_format=['next_word'])

# Reduced sentence quality

'Hei, olen Rossi.'

In [182]:
l = L.LLM(f'You are a Finnish language teacher named Rossi.')
out = l(f'Is this sentence correct: Tama on minun koirani', response_format='stream')
for t in out: print(t, end='')

l('breifly summarize whats wrong with minun')

Hyvä kysymys! (Good question!)

The sentence "Tama on minun koirani" is almost correct, but not quite.

In Finnish, the correct sentence would be "Tämä on minun koirani".

The difference is in the word "tama" vs. "tämä". "Tämä" is the correct word to use when referring to something specific, like "this" in English. "Tama" is not a word in Finnish.

So, "Tämä on minun koirani" means "This is my dog".

'In Finnish, when you\'re talking about something belonging to you, you don\'t need to use the word "minun". Instead, you can use the possessive suffix "-ni" attached to the noun.\n\nSo, instead of "minun koirani", you would say "koirani", which means "my dog".\n\nSo, the corrected sentence would be: "Tämä on koirani".'

In [173]:
l = L.LLM(f'You are a Finnish language teacher named Rossi.\n\nYour responses must only use words in this allowed vocab: {allowed_words} and any emoji/punctuation.')
out = l(f'onko sinulla makea herkku', response_format='stream')

words = []
word = ''
for t in out:
    print(f'{t}', end='')

    # if t.startswith(' '):
    #     if word:
    #         words.append(word)
    #         # print(word)
    #     word = t[1:]
    # elif t.endswith(',') or t.endswith('!') or t.endswith('.'):
    #     word += t[:-1]
    #     # print(word)
    #     words.append(word)
    #     word = ''
    # else:
    #     word += t

words

Kyllä! Minä olen Rossi. 🐶 Koira on mukava.

[]

In [111]:
l._hist

[Msg(role='system', content='You are a Finnish language teacher named Rossi.', response_format=None),
 Msg(role='user', content="Give me a finnish sentence. This is your allowed vocab: ['hei', 'olen', 'suomi', 'rossi']. Only output the sentence. ", response_format=None),
 Msg(role='assistant', content='Hei, olen Rossi ja opetan suomea.', response_format=None)]